# Two programs:
$1.$ print out all d-seperations of a given DAG  

$2.$ take $P_{AB|XY}$ return if compatible w/ bell DAG, can be done by  
* checking if can find the product distribution that fit Bell DAG (does there exist a structural eqn?)  
* use Fine's theorem. can the product of distributions be found using the convex combination of the 16 datatables?
(f(x->a))... each w/ bunch
* do this w/ any cardinality i.e. for two binary input output we can have 4 funcitons:  
    $f(a,b) = (0,0)\\
    f(a,b) = (1,1)\\
    f(a,b) = (b,a)\\
    f(a,b) = (a,b)$  
    gives us 4 possible functions, set up program for any cardinality.

## Structural Equation Test to the Bell DAG:

given known values of $P_{ABC}(a,b,c)$ for all binary permutations of $A,B,C$:  
$P_{ABC}(a,b,c) =  \sum P_\lambda (\lambda) P_{A|\lambda}(a, \lambda) P_{B|A}(b,a) P_{C|B \lambda}(c,b,\lambda) \qquad \forall \lambda \in [0,card]$  
we find set of values of $P_\lambda (\lambda), \space P_{A|\lambda}(a, \lambda), \space P_{C|B \lambda}(c,b,\lambda)$ that match this constraints, we then find the set of three values that maximize $P_{C |do(B)}(C,B)$


note: can change the cardinality of the hidden variable lamda.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import itertools
import numpy as np

In [ ]:
# Alice and Bob's settings
card_X = 2
card_Y = 2

# Alice and Bob's results
card_A = 2
card_B = 2

# hidden common cause
card_l = 2

In [ ]:
observable_probs = np.arange(card_A * card_B * card_X * card_Y)
observable_probs = observable_probs / observable_probs.sum()
observable_probs = observable_probs.reshape(card_A, card_B, card_X, card_Y)

# defining a distribution P(AB|XY) based on cardinality of X, Y, A, B
dist_ABXY = {}
for a, b, x, y in itertools.product(range(card_A), range(card_B), range(card_X), range(card_Y)):
    prob = observable_probs[a, b, x, y]
    dist_ABXY[(a, b, x, y)] = prob


In [ ]:
dist_ABXY

distribution feasible?

In [ ]:
sum(dist_ABXY.values()) == 1

In [ ]:
# get P_AB|XY from P_ABXY
def P_AB_giv_XY(A,B, X, Y):
    P_ABXY = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B and x==X and y==Y])
    P_AB = sum([dist_ABXY[(a,b,x,y)] for a,b,x,y in dist_ABXY if a==A and b==B])


    return P_ABXY/P_AB

sanity check for feasibility:


In [1]:
import numpy as np 
from qutip import *
import itertools

def probEJM(V1,V2,theta):
    #sources
    psi_ = bell_state(state='11') # this is |psi->
    psi = ket2dm(psi_) #this is |psi-><psi-|
    iden2 = sigmax()*sigmax()
    iden4 = tensor(iden2,iden2)
    rho1 = V1*psi + ((1-V1)/4)*iden4
    rho2 = V2*psi + ((1-V2)/4)*iden4
    rho = tensor(rho1,rho2)
    #measurements
    A=np.zeros(24,dtype = 'complex_').reshape([2,2,3,2]) # initialization of an hypermatrix A the last to indices corresponds to x and a respectively, i.e. A[:,:,0,1] is the measurement done when x=0 and a=-1 (en el ultimo indice 0 corresponde a 1 y 1 a -1)
    C=np.zeros(24,dtype = 'complex_').reshape([2,2,3,2]) # the same for C
    A1 = sigmax() # measurement of A for x = 1
    A2 = sigmay() # measurement of A for x = 2
    A3 = sigmaz() # measurement of A for x = 3
    # A1 = (sigmax()+sigmax())/np.sqrt(2) # measurement of A for x = 1
    # A2 = sigmay() # measurement of A for x = 2
    # A3 = (sigmax()-sigmax())/np.sqrt(2) # measurement of A for x = 3
    eival_A1,eivect_A1 = A1.eigenstates() #eigen of A for x = 1
    eival_A2,eivect_A2 = A2.eigenstates() #eigen of A for x = 2
    eival_A3,eivect_A3 = A3.eigenstates() #eigen of A for x = 3
    A[:,:,0,0] = np.dot(eivect_A1[0],eivect_A1[0].dag()) # measurement of A for x = 1 corresponding to a=1
    A[:,:,0,1] = np.dot(eivect_A1[1],eivect_A1[1].dag()) # measurement of A for x = 1 corresponding to a=-1
    A[:,:,1,0] = np.dot(eivect_A2[0],eivect_A2[0].dag())
    A[:,:,1,1] = np.dot(eivect_A2[1],eivect_A2[1].dag())
    A[:,:,2,0] = np.dot(eivect_A3[0],eivect_A3[0].dag())
    A[:,:,2,1] = np.dot(eivect_A3[1],eivect_A3[1].dag())
    C = A
    # Elegant joint measurement for Bob
    B = np.zeros((4,4,1,4),dtype = 'complex_')
    eta = (1/np.sqrt(3))*np.array([1,-1,-1,1])
    phib = (np.pi/4)*np.array([1,-1,3,-3])
    zero = basis(2)
    one = basis(2,1)
    m = np.zeros((2,1,4),dtype = 'complex_')
    m_ = np.zeros((2,1,4),dtype = 'complex_')
    phi_theta = np.zeros((4,1,4),dtype = 'complex_')
    for b in range(4):
        m[:,:,b] = np.sqrt((1+eta[b])/2)*np.exp(-1j*phib[b]/2)*zero + np.sqrt((1-eta[b])/2)*np.exp(1j*phib[b]/2)*one
        m[:,:,b] = Qobj(m[:,:,b])
        m_[:,:,b] = np.sqrt((1-eta[b])/2)*np.exp(-1j*phib[b]/2)*zero - np.sqrt((1+eta[b])/2)*np.exp(1j*phib[b]/2)*one
        m_[:,:,b] = Qobj(m_[:,:,b])
        phi_theta[:,:,b] = ((np.sqrt(3)+np.exp(1j*theta))/(2*np.sqrt(2)))*np.kron(m[:,:,b],m_[:,:,b]) + ((np.sqrt(3)-np.exp(1j*theta))/(2*np.sqrt(2)))*np.kron(m_[:,:,b],m[:,:,b])
        B[:,:,0,b] = np.dot(phi_theta[:,:,b],phi_theta[:,:,b].conj().T)

    prob = np.zeros((2,4,2,3,1,3))
    for a,b,c,x,y,z in itertools.product(*[range(i) for i in (2,4,2,3,1,3)]):
        prob[a,b,c,x,y,z] = np.trace(np.dot(rho,np.kron(A[:,:,x,a],np.kron(B[:,:,y,b],C[:,:,z,c]))))
    return prob

print(probEJM(1,1,0))


c:\Users\dvirz\AppData\Local\Programs\Python\Python310\lib\site-packages\qutip\__init__.py:24: UserWarning: matplotlib not found: Graphics will not work.
  warnings.warn("matplotlib not found: Graphics will not work.")


TypeError: must be real number, not Qobj

$P_Z(Z)$

In [ ]:
# P_A(a) = sum_{B,C} P_ABC(a,b,c)
def P_A(a):
    return sum([dist[(a, B, C)] for B in range(card_B) for C in range(card_C)])

$P_\lambda (\lambda), \space P_{A|\lambda} (A, \lambda), \space P_{C|B,\lambda} (C, B, \lambda)$

In [ ]:
m = gp.Model()
#m.Params.LogToConsole = 0

# variables (MVars)
P_l = m.addMVar(shape = card, vtype=GRB.CONTINUOUS, name="P_l", lb=0, ub=1)
P_C_given_B_l = m.addMVar(shape = (card_C, card_B, card), vtype=GRB.CONTINUOUS, name="P_C_given_B_l", lb=0, ub=1)

# instrumental DAG:
P_B_given_A_l = m.addMVar(shape = (card_B, card_A, card), vtype=GRB.CONTINUOUS, name="P_A_given_l", lb=0, ub=1)


## can't add products of three variables, so we add a new helper variable
tripple_prod = m.addMVar(shape=(card_C, card_B, card, card), vtype=GRB.CONTINUOUS, name="c do b lamdbda time p_lambda", lb=0, ub=1) 
p_C_do_B = m.addMVar(shape=(card_C, card_B), vtype=GRB.CONTINUOUS, name="c do b", lb=0, ub=1)
m.update()

for a, b, c in itertools.product(range(card_A), range(card_B), range(card_C)):
    P_ABC = dist[(a, b, c)]
    RHS_obs = gp.LinExpr()
    RHS_do = gp.LinExpr()    
    for l in range(card):
        m.addConstr(tripple_prod[b, c, l] == P_l[l] * P_C_given_B_l[c, b, l])
        ##############
        #if instrumental:
        RHS_obs += tripple_prod[b, c, l]*P_B_given_A_l[b, a, l]*P_A(a) # instrumental DAG
        ##############
        RHS_do += tripple_prod[b, c, l]
    # probability distribution to symbolic equations equality constraint 
    m.addConstr(P_ABC == RHS_obs)
    m.addConstr(p_C_do_B[c, b] == RHS_do)


for l in range(card):
    m.addConstr(gp.quicksum(P_l[l] for l in range(card)) == 1, "sum_P_l = 1")
    m.addConstr(gp.quicksum(P_A_given_l[a, l] for a in [0, 1]) == 1, f"sum_P_A_given_l_{l} = 1")
    
    # only in card_B = 2
    m.addConstr(gp.quicksum(P_C_given_B_l[c, 0, l] for c in [0, 1]) == 1, f"sum_P_C_given_B_l_0_{l} = 1")
    m.addConstr(gp.quicksum(P_C_given_B_l[c, 1, l] for c in [0, 1]) == 1, f"sum_P_C_given_B_l_1_{l} = 1")

    m.addConstr(gp.quicksum(P_B_given_A_l[b, 0, l] for b in [0, 1]) == 1, f"sum_P_C_given_B_l_0_{l} = 1")
    m.addConstr(gp.quicksum(P_B_given_A_l[b, 1, l] for b in [0, 1]) == 1, f"sum_P_C_given_B_l_1_{l} = 1")

In [ ]:
# can take (range(card_C), range(card_B), "max" or else is min) as input
def main(c,b):
    print(f"optimizing P(C={c}|do(B={b}))...")
    m.setObjective(p_C_do_B[c,b], GRB.MINIMIZE)
    m.optimize()
    min_val = p_C_do_B[c,b].X.item()

    m.setObjective(p_C_do_B[c,b], GRB.MAXIMIZE)
    m.optimize()
    max_val = p_C_do_B[c,b].X.item()

    print("\nmin value: ", min_val)
    print("max value: ", max_val)
    print("distance:", max_val - min_val)

$\text{max/min }(P_{C|\text{do}(B)})$ over the three unknowns.

In [ ]:
main(1,1)